# Crown

This notebook that analysis the Crown.

---------------

##### Imports

In [ ]:
# Uncomment to load the local package rather than the pip-installed version.
# Add project src to path.
import set_path

In [ ]:
# Import modules.
import os
import math
import time
import trimesh
import shapely
import pymeshfix
import subprocess
import alphashape
import numpy as np
import open3d as o3d
import logging as log
import networkx as nx
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.spatial import KDTree
from descartes import PolygonPatch
from shapely.geometry import Polygon
from scipy.spatial import ConvexHull
from plyfile import PlyData, PlyElement
from skimage.measure import CircleModel, ransac
from sklearn.neighbors import NearestNeighbors
import utils.math_utils as math_utils
import utils.plot_utils as plot_utils
import utils.o3d_utils as o3d_utils
from misc.quaternion import Quaternion

In [ ]:
class Crown:
    """ This class implements the AdTree delft repository.
    Attributes:
        ---
    """

    def __init__(self):
        pass

    def model(self, pcd):
        pass

##### Experimental

In [ ]:
# Load point cloud data
source = 'x'

if source == 'ahn':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_AHN.las')
elif source == 'cyclo':
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Cyclo.las')
else:
    pcd = o3d_utils.read_las('../datasets/single_selection/single_121913_487434_Sonarski.las')

In [ ]:
stem_cloud = pcd.select_by_index(np.where(np.asarray(pcd.points)[:,2]>5)[0])
o3d.visualization.draw_geometries([stem_cloud])

In [ ]:
crownAnalysis = Crown()

In [ ]:
meshes, stats = crownAnalysis.model(stem_cloud)